In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import math
import csv
import os

## ILP for the Unbounded Multiple Knapsack Problem with Processing Times

In [2]:
n = 10 # number of campaigns
m = 1000 # number of customers
d = 45 # days to be scheduled

s = np.random.randint(d-1, size = n) # starting time
p = np.random.choice([5, 8, 15, 17, 23, 30, 60], n) # processing time
c = np.random.randint(low = 10, high = 40, size = n) # cost
b = np.random.randint(low = 40, high = 80, size = n) # benefit
C = np.random.randint(low = 40, high = 100, size = n) # maximum cost

zippedList =  list(zip(s, p, c, b, C))
campaigns = pd.DataFrame(zippedList, columns = ['starting time', 'processing time' , 'cost', 'benefit', 'maximum cost'])
campaigns.index.names = ['name']
# print("Dataframe : " , campaigns, sep='\n')

# matrix with take-rates
T = np.random.random_sample((m, n))

In [4]:
# create empty model
model = gp.Model()

# add decision variables
x = model.addMVar((m,n), vtype=GRB.BINARY, name='x') # variable x_ij
z = model.addMVar((m,n,d), vtype=GRB.BINARY, name = 'z') # variable z_ijl
 
# set objective function
model.setObjective((sum((b[j]*T[i][j] - c[j]) * x[i,j] for j in range(n) for i in range(m))), GRB.MAXIMIZE)

# add constraints
model.addConstrs(((sum(c[j]*x[i,j] for i in range(m))) <= C[j]) for j in range(n))

model.addConstrs(((sum(z[i,j,l] for j in range(n))) <= 1) for i in range(m) for l in range(d))

model.addConstrs(((s[j]+p[j])*x[i,j] <= d) for i in range(m) for j in range(n))

model.addConstrs((z[i,j,l] == 0) for j in range(n) for i in range(m) for l in range(d) if l not in range(s[j],min(s[j]+p[j],d)))

model.addConstrs(((sum(z[i,j,l] for l in range(s[j],min(s[j]+p[j],d)))) == p[j]*x[i,j]) for j in range(n) for i in range(m))

# solve model
model.optimize()

# get attributes
if model.SolCount > 0:
    x_values = []
    for k in range(m*n):
        current = model.getVarByName("x[{}]".format(k))
        x_values.append((current.varName, current.x))
    
cwd = os.getcwd()
path = '{}/output'.format(cwd)
if not os.path.exists(path):
    os.makedirs(path)
    
# write schedule into file
with open('{}/ptumkp.csv'.format(path), mode = 'w') as schedule:
    writer = csv.writer(schedule, delimiter = ';')
    writer.writerow(['campaign','customer','starting time', 'processing time'])

    for value in x_values:
        if value[1] == 1.0:
            count = int(value[0][value[0].find('[')+1:value[0].find(']')])
            customer = math.floor(count/n)
            campaign = count%n
            writer.writerow(campaign, customer, s[campaign], p[campaign])

schedule.close()

# export model
# model.write('ptumkp.lp')

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license